In [5]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql

import func_mod as funciones  ###archivo de funciones propias
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import metrics

import numpy as np
import matplotlib.pyplot as plt ### gráficos
from sklearn.model_selection import RandomizedSearchCV
import joblib  ### para guardar modelos
from sklearn.preprocessing import StandardScaler ## escalar variables 
import openpyxl
from sklearn.inspection import permutation_importance # Para determinar imporancia de variables
from sklearn.model_selection import train_test_split # Particion de datos de entrenamiento y testeo

In [6]:
# Cambiar ruta a ruta local de cada usuario
df = pd.read_excel(r'C:\Users\envy\Desktop\Proyectos VS code\Proyecto analitica III\Proyecto-Analitica-3\data\base_a_modelar.xlsx')

In [7]:
# Variable respuesta
y=df.Attrition

# Variables explicativas
X= df.loc[:,~df.columns.isin(['Attrition','EmployeeID'])]

In [8]:
param_grid = {
    'n_estimators': [10, 25, 50, 75, 100],  # Número de árboles
    'max_depth': [5,10,15,20],    # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4]
}

# Llamada a la función
results_df, best_rf_model = funciones.evaluate_rf_hyperparameters(X, y, param_grid)

# Visualizar resultados
print(results_df)

Fitting 5 folds for each of 180 candidates, totalling 900 fits
                                         best_params  precision   recall  \
0  {'max_depth': 20, 'min_samples_leaf': 1, 'min_...   0.882353  0.78125   

         f1    aucpr  
0  0.828729  0.82269  


In [9]:
results_df

,best_params,precision,recall,f1,aucpr
0,"{'max_depth': 20, 'min_samples_leaf': 1, 'min_...",0.882353,0.78125,0.828729,0.82269


In [10]:
best_rf_model

RandomForestClassifier(max_depth=20, n_estimators=75, random_state=42)

In [15]:
# Verificacion de importancia de variables
# Se usa de la misma manera en la funcion evaluate_rf_hyperparameters del paquete de funciones propias func_mod.py
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Calcular la importancia de las características mediante permutación
result = permutation_importance(best_rf_model, X_test, y_test, n_repeats=10, random_state=42)


# Mostrar la importancia de las variables
for feature, importance in zip(X.columns, result.importances_mean):
    print(f"{feature}: {importance}")

Age: 0.020697674418604615
DistanceFromHome: 0.005426356589147252
Education: 0.0025581395348836635
JobLevel: 0.0024806201550387043
MonthlyIncome: 0.008449612403100737
NumCompaniesWorked: 0.010232558139534841
PercentSalaryHike: 0.00325581395348834
StockOptionLevel: 0.002248062015503827
TotalWorkingYears: 0.0217054263565891
TrainingTimesLastYear: 0.0006201550387596622
YearsAtCompany: 0.005736434108527111
YearsSinceLastPromotion: 0.007674418604651134
YearsWithCurrManager: 0.007596899224806175
JobInvolvement: -0.0009302325581395876
PerformanceRating: 7.751937984495916e-05
EnvironmentSatisfaction: 0.008527131782945696
JobSatisfaction: 0.007209302325581368
WorkLifeBalance: 0.0013178294573643057
BusinessTravel_Non-Travel: 0.00023255813953484416
BusinessTravel_Travel_Frequently: 0.008062015503875952
BusinessTravel_Travel_Rarely: -7.751937984501467e-05
Department_Human Resources: 0.0031782945736433367
Department_Research & Development: -4.4408920985006264e-17
Department_Sales: -0.001085271317829

Valores positivos:

Un valor positivo significa que, al permutar esa característica, el rendimiento del modelo empeoró (se redujo la precisión, aumentó el error). Por lo tanto, la variable tiene un impacto positivo en el rendimiento del modelo, es decir, es importante.

Age: 0.020697674418604615
Esto significa que al permutar la variable Age, el rendimiento del modelo se deterioró en aproximadamente 0.0207 unidades (posiblemente en términos de precisión).

Valores cercanos a cero:
Los valores cercanos a cero indican que la permutación de esa característica no tiene mucho impacto en el rendimiento del modelo, lo que significa que la variable es menos importante.

PerformanceRating: 7.751937984495916e-05
Este valor es casi nulo, lo que sugiere que la variable PerformanceRating tiene poco o ningún impacto en el rendimiento del modelo.
Valores negativos:

Los valores negativos indican que, al permutar esa característica, el rendimiento del modelo mejoró ligeramente, lo cual es inusual pero puede ocurrir cuando una característica introduce ruido o información redundante.

JobRole_Research Director: -0.0007751937984497026
Este valor negativo indica que, al permutar la variable JobRole_Research Director, el rendimiento del modelo mejoró un poco, lo que sugiere que esta característica puede no ser útil para el modelo o incluso puede estar añadiendo ruido.
Interpretación práctica:
Características importantes: Variables con mayores valores de importancia positiva, como:

Age: 0.0207
TotalWorkingYears: 0.0217
Estas variables tienen un gran impacto en el rendimiento del modelo y son fundamentales para las predicciones.

Características menos importantes: Variables con valores cercanos a cero o negativos, como:

JobRole_Research Director: -0.0007751
EducationField_Medical: -0.000155
Estas variables tienen muy poco o ningún impacto en las predicciones del modelo, y en algunos casos, su permutación podría mejorar el rendimiento, lo que sugiere que no aportan valor o podrían ser irrelevantes.

In [ ]:
# Obtén las predicciones del mejor modelo para el conjunto de prueba
y_pred = best_rf_model.predict(X_test)

# Si deseas verlas junto con las probabilidades:
y_scores = best_rf_model.predict_proba(X_test)[:, 1]

# Crear un DataFrame con las predicciones y las probabilidades
predictions_df = pd.DataFrame({
    'true_values': y_test,  # Valores reales
    'predicted_values': y_pred,  # Predicciones
    'predicted_probabilities': y_scores  # Probabilidades
})

# Mostrar las predicciones y probabilidades
print(predictions_df)